# REINFORCE ALGORITHM
In this notebook, we will implement the Reinforce algorithm also known as the Monte Carlo Policy Gradient. To test its robustness, we would train it in a simple environment e.g. Cartpole-v1. The simple environment is chosen to be sure that the algorithm works before going deeper. Since finding bugs would be much easier in simple environments. This implementation is based on [Hugging Face Deep RL Course](https://huggingface.co/learn/deep-rl-course/unit4/introduction).


<img src="https://www.researchgate.net/profile/Kan-Zheng-2/publication/334623894/figure/fig1/AS:783667001565185@1563852181878/Reinforcement-learning-process.png" alt="Reinforcement Learning"/>

Diagram illustrating the system behind Reinforcement learning in contrast to the diagram below illustrating the policy optimization or policy-based learning.

<img src="https://static.packt-cdn.com/products/9781788991063/graphics/assets/e760c7dd-ebb7-4f86-80a1-99b79ca3f002.png" alt="Policy-based learning"/>

## Introduction
Reinforce is a Policy-based method i.e. a deep reinforcement learning algorithm that tries to optimize the policy directly without using an action-value function. More precisely, Reinforce is a Policy-gradient method, a subclass of Policy-based methods that aims to optimize the policy directly by estimating the weights of the optimal policy using gradient ascent. A policy is generated directly from the state-value function. The policy gradient algorithms search for a local maximum in the state value function by ascending the gradient of the policy w.r.t the parameters.

## The CartPole-v1 environment
> A pole is attached by an un-actuated joint to a cart, which moves along a frictionless traack. The pendulum is placed upright on the cart and the goal is to balance the pole by applying forces in the left and right direction on the cart. The goal is to push the cart left or right **so that the pole stays in the equilibruim**

The episode ends if:
- The pole Angle is greater than ±12°
- Cart Position is greater than ±2.4
- Episode length is greater than 500

A reward of +1 is received in each timestep as long as the Pole stays in the equilibrium.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/reinforce.png" alt="Reinforce"/>

## REINFORCE ARCHITECTURE
This is the pseudocode of the REINFORCE algorithm

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/pg_pseudocode.png" alt="Reinforce algorithm"/>

### 1. Import the necessary libraries   

### 2. Define the architecture of the policy

### 3. Train the agent with REINFORCE

### 4. Plot the Scores

### 5. Watch the Agent